# Explainable model for predicting heart failure using the QLattice

The QLattice is a supervised machine learning tool for symbolic regression developed by [Abzu](https://www.abzu.ai) . It is inspired by Richard Feynman's path integral formulation. That's why the python module to use it is called *Feyn*, and the *Q* in QLattice is for Quantum.

Abzu provides free QLattices for non-commercial use to anyone. These free community QLattices gets allocated for you automatically if you use Feyn without an active subscription, as we will do in this notebook. Read more about how it works here: https://docs.abzu.ai/docs/guides/getting_started/community.html

The feyn Python module is not installed on Kaggle by default so we have to pip install it first. 

__Note__: the pip install will fail unless you enable *Internet* in the *settings* to the right--->

In [ ]:
!pip install feyn

# Python imports
In this notebook we will use only two python modules: the `feyn` module to access the QLattice, the `pandas` module to access the data, and `sklearn` to split the data into train and test sets

In [ ]:
import feyn
import pandas as pd
import sklearn.model_selection

# Getting the Data
First let's load the dataset and take a look

In [ ]:
data = '/kaggle/input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv'
df = pd.read_csv(data)
df

In [ ]:
df.isna().sum()

# First impressions:
We notice that:
- The target variable is `DEATH_EVENT`
- All data types are numerical. The QLattice works with both categorical and numerical data, but needs to be told which entries are categorical (i.e. it assumes they are numerical). Since all entries in this dataset are numerical we're all good!
- There are no missing entries

Let's remove the `time` column

In [ ]:
df.drop(columns={'time'}, axis=1, inplace=True)

# Target variable
Let's take a look at the distribution of the target variable (how many people died versus survived)

In [ ]:
df.DEATH_EVENT.value_counts()

We can see that it is slightly skewed towards survival
# Splitting the data
Let's split the data into train and test sets. We will stratify by `DEATH_EVENT` and take 2/3 of the entire dataset for training

In [ ]:
train, test = sklearn.model_selection.train_test_split(df, stratify=df["DEATH_EVENT"], train_size=.66, random_state=1)

In [ ]:
test.DEATH_EVENT.value_counts()

# Allocate a QLattice
The actual QLattice is a quantum simulator that runs on Abzu's hardware, but we can allocate one with a single line of code. Cool, huh?

In [ ]:
ql = feyn.connect_qlattice()

# Resetting and reproducability
The QLattice has the potential to store learnings between sessions to enable transfer of learning and federated learning. This is not possible with Community QLattices, since a new one gets allocated whenever we run the notebook, so it is not strictly necessary to call the reset function on our new QLattice.

But the reset function also allows us to provide a random seed, which will ensure that we get the same results every time we run this notebook

In [ ]:
ql.reset(random_seed=1)

# Search for the best model
We are now ready to instruct the QLattice to search for the best mathematical model to explain the data. Here we use the high-level convenience function that does everything with sensible defaults: https://docs.abzu.ai/docs/guides/essentials/auto_run.html.

For more detailed control, we could use the primitives: https://docs.abzu.ai/docs/guides/primitives/using_primitives.html

NOTE: This will take a minute to complete. It invoves work done on the QLattice machine remotely as well as in the local notebook. The part that runs locally is slowing things down because of the limited CPU resources on Kaggle. Running the same on my machine locally only takes 10 seconds!

In [ ]:
ql.reset(random_seed=1)
models = ql.auto_run(train, output_name="DEATH_EVENT", kind="classification", max_complexity=10, criterion='aic')

# What did we find?
`models` is a list of graphs sorted by accuracy. Each model shows how the selected features, or inputs, interact to achieve the output. We can access the best graph by calling:

In [ ]:
models[0]

# Performance on train vs. test
Let's see how the model performs on the train versus test dataset. We are looking for high accuracy and AUC, but similar values across the two datasets (we don't want overfitting!)

In [ ]:
models[0].plot(train, test)

# Comparaison
Let's compare this to three other Machine Learning algorithms: Random Forest, Gradient Boost, and Logistic Regression

In [ ]:
rf = feyn.reference.RandomForestClassifier(train, output_name="DEATH_EVENT")
gb = feyn.reference.GradientBoostingClassifier(train, output_name="DEATH_EVENT")
lr = feyn.reference.LogisticRegressionClassifier(train, output_name="DEATH_EVENT", max_iter=10000)

We can visualize their relative performances using their respective ROC curves

In [ ]:
models[0].plot_roc_curve(test, label='QLattice')
rf.plot_roc_curve(test, label="Random Forest")
gb.plot_roc_curve(test, label="Gradient Boosting")
lr.plot_roc_curve(test, label="Logistic Regression")

We can see here that the QLattice outperforms Gradient Boost and Logistic Regression. The ROC curve for the QLattice and Random Forest seem to resemble one another. So which is better?

# QLattice vs. Random Forest: type of failure matters!
Let's take a closer look at how the two models interact with the data. The performances of the two models can be quickly evaluated using accuracy or AUC, however, important insight as to which is better can be found by looking at **where the models fail**.

First let's ask ourselves the question: Which is worse? Having the doctor tell you that you will experience heart failure and then not, or being told you are fine and dying? Clearly the second! A model predicting that you will be fine, but actually experiencing heart failure is called a false negative. In medical prediction cases such as this false negatives can be life threatening. Therfore when evaluating model performance we want to chose one that minimizes this outcome. We can see the number of false negative predictions the QLattice and Random Forest models make using a **confusion matrix**.

In [ ]:
models[0].plot_confusion_matrix(test)


In [ ]:
rf.plot_confusion_matrix(test)

We can see here that the QLattice model has a much lower false negative rate. It predicts that 13 heart failure subjects are fine compared to the 19 predicted by Random Forest. From a medicine perspective, the QLattice model is therefore better.

# What did we learn?
1. Interestingly, with only four inputs: ejection fraction, serum creatinine, sex, and age, we can predict whether or not someone will die from heart failure relatively well. Cooler still, ejection fraction and serum creatinine have previously been identified as important features using other machine learning algorithms as cited by the authors of the original dataset. From this we can see that **the QLattice is an effective feature selection tool**.

2. The QLattice also creates models that show not only which features interact to produce the outputs, but also **how** they relate to one another.

3. In terms of pure AUC the QLattice model performs better than (Gradient Boost and Logistic Regression) or similar to (Random Forest) other machine learning algorithms.

4. As an overall application in medecine, the QLattice model is the best option because it combines a high AUC score with a lower false negative rate when compared to its biggest competator, Random Forest.



**Pretty neat!**